In [1]:
import pandas as pd
import pyodbc
import os
import warnings

In [2]:
# Conectarse a Azure SQL
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=sql-server-mamba.database.windows.net;'
    'DATABASE=sql-db-mamba;'
    'UID=sql-admin;'
    'PWD=serverpwd1!'
)

In [ ]:
def clean_value(val):
    if pd.isna(val):
        return None
    if isinstance(val, float):
        return round(val, 6)
    return val


cursor = conn.cursor()
for table_file in os.listdir():
    if not (table_file.endswith('.csv') or table_file.endswith('.xlsx')):
        continue
    table_name = os.path.splitext(table_file)[0]
    print(f"Inserting table {table_name}...")
    df = pd.read_csv(table_file) if table_file.endswith('.csv') else pd.read_excel(table_file)
    columns = df.columns.tolist()
    placeholders = ', '.join(['?'] * len(columns))
    column_names = ', '.join(columns)

    insert_query = f"INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})"
    cleaned_rows = (
        tuple(clean_value(val) for val in row)
        for row in df.itertuples(index=False, name=None)
    )
    cursor.executemany(insert_query, cleaned_rows)
    conn.commit()
cursor.close()

Inserting table Cuadriculas...
Inserting table CuadriculasEspecies...
Inserting table CuadriculasRutas...
Inserting table DescripcionesEspecies...
Inserting table NombresEspecies...
Inserting table Rutas...
Inserting table Taxonomia...


In [4]:
warnings.filterwarnings('ignore')
for table_file in os.listdir():
    if not (table_file.endswith('.csv') or table_file.endswith('.xlsx')):
        continue
    table_name = os.path.splitext(table_file)[0]
    df_sql = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    df_file = pd.read_csv(table_file) if table_file.endswith('.csv') else pd.read_excel(table_file)
    print(f"Table {table_name} has {len(df_sql)} records and file {table_file} has {len(df_file)} records")
warnings.filterwarnings('default')

Table Cuadriculas has 17939 records and file Cuadriculas.csv has 17939 records
Table CuadriculasEspecies has 857733 records and file CuadriculasEspecies.csv has 857733 records
Table CuadriculasRutas has 11732 records and file CuadriculasRutas.csv has 11731 records
Table DescripcionesEspecies has 2776 records and file DescripcionesEspecies.xlsx has 2776 records
Table NombresEspecies has 13552 records and file NombresEspecies.csv has 13552 records
Table Rutas has 4645 records and file Rutas.xlsx has 4645 records
Table Taxonomia has 3896 records and file Taxonomia.xlsx has 3896 records


In [5]:
conn.close()